# Using the H$\delta_A$ and $D_n 4000\,Å$ break indices to quantify the "burstiness" of the SFH
---
Kauffmann et al. (2003) found that stellar continua with "bursty" and continunous SFHs can be distinguished by plotting the H$\delta_A$ index against the $D_n 4000\,Å$ break strength. 

In this notebook, we will 
* Define functions for computing both stellar indices;
* use the Gonzalez-Delgado stellar templates to create "bursty" and continuous SFHs;
* Measure the H$\delta_A$ and $D_n 4000\,Å$ break strengths of these spectra, and recreate their Figs. 2 and 3, to check that the same relationships exist with our templates;
* Investigate the effects of an AGN continuum on these figures;
* Compute these indices for Phil's spectra and see if they lie in the expected regions of the H$\delta_A$ vs. $D_n 4000\,Å$ break strength diagram;
* Using ppxf, remove emission lines from noisy mock spectra & check that the indices can accurately be recovered.

In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [73]:
import numpy as np
import os
from astropy.io import fits
from scipy import constants
from tqdm.notebook import tqdm
import pandas as pd
from itertools import product

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, convert_mass_weights_to_light_weights
from ppxftests.sfhutils import compute_mw_age, compute_lw_age, compute_sfr_thresh_age, compute_sb_zero_age, compute_mass
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted

from IPython.core.debugger import Tracer

import matplotlib.pyplot as plt 
plt.ion()
plt.close("all")

In [4]:
###################################################
# Load SSP parameters
###################################################
bin_edges, bin_widths = get_bin_edges_and_widths(isochrones="Padova")
_, _, metallicities, ages = load_ssp_templates(isochrones="Padova")
N_metallicities = len(metallicities)
N_ages = len(ages)

In [5]:
###################################################
# Function for computing the HdeltaA index 
# as per Worthey & Ottaviani (1997)
###################################################
def compute_Hdelta(spec, lambda_vals_A, z):
    # De-redshift the input spectrum
    lambda_vals_rest_A = lambda_vals_A / (1 + z) 

    # Define the blue & red passbands, plus the midpoint
    lambda_start_b_A = 4041.60
    lambda_stop_b_A = 4079.75
    lambda_mid_b_A = 0.5 * (lambda_start_b_A + lambda_stop_b_A)
    lambda_start_r_A = 4128.50 
    lambda_stop_r_A = 4161.00
    lambda_mid_r_A = 0.5 * (lambda_start_r_A + lambda_stop_r_A)
    
    # Central passband
    lambda_start_c_A = 4083.50 
    lambda_stop_c_A = 4122.25
    lambda_mid_c_A = 0.5 * (lambda_start_c_A + lambda_stop_c_A)

    # Determine the corresponding indices in the input spectrum
    lambda_start_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_b_A))
    lambda_stop_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_b_A)) 
    lambda_start_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_r_A))
    lambda_stop_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_r_A)) 
    lambda_mid_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_mid_b_A)) 
    lambda_mid_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_mid_r_A)) 
    lambda_start_c_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_c_A))
    lambda_stop_c_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_c_A)) 

    # Compute the mean flux in the blue & red passbands 
    f_mean_b = np.nanmean(spec[lambda_start_b_idx:lambda_stop_b_idx])
    f_mean_r = np.nanmean(spec[lambda_start_r_idx:lambda_stop_r_idx])

    # Define the continuum within the central passband to be halfway between the mean fluxes in the red & blue passbands 
    f_mean_c = 0.5 * (f_mean_b + f_mean_r)

    # Integrate the spectrum from lambda_start_c_A to lambda_stop_c_A to compute the EW
    dlambda_rest_A = lambda_vals_rest_A[1] - lambda_vals_rest_A[0]
    A_spec = dlambda_rest_A * np.nansum(spec[lambda_start_c_idx:lambda_stop_c_idx])

    # Integrate the corresponding area under the pseudocontinuum
    m = (f_mean_b - f_mean_r) / (lambda_mid_b_A - lambda_mid_r_A)
    b = f_mean_b - m * lambda_mid_b_A
    y = lambda l: m * l + b
    A_trapezoid = 0.5 * (y(lambda_start_c_A) + y(lambda_stop_c_A)) * (lambda_stop_c_A - lambda_start_c_A)

    # Calculate area in between the pseudocontinuum and the spectrum!! 
    A_line = A_trapezoid - A_spec
    
    # Compute the EW, taking the level of the pseudocontinuum in the centre of the passband to be the continuum level
    EW_A = A_line / f_mean_c
    
    # ###################################################
    # # Plot
    # ###################################################
    # fig, ax = plt.subplots(figsize=(15, 5))
    # ax.plot(lambda_vals_rest_A, spec, color="grey")
    # ax.axvline(lambda_start_b_A, color="blue", label="Blue passband")
    # ax.axvline(lambda_stop_b_A, color="blue")
    # ax.axvline(lambda_start_r_A, color="red", label="Red passband")
    # ax.axvline(lambda_stop_r_A, color="red")
    # ax.axvline(lambda_start_c_A, color="grey", label="Central passband")
    # ax.axvline(lambda_stop_c_A, color="grey")
    # ax.plot([lambda_start_b_A, lambda_stop_b_A], [f_mean_b, f_mean_b], linewidth=3, color="black", label="Mean flux in blue/red passband")
    # ax.plot([lambda_start_r_A, lambda_stop_r_A], [f_mean_r, f_mean_r], linewidth=3, color="black")
    # ax.plot([lambda_mid_b_A, lambda_mid_r_A], [f_mean_b, f_mean_r], ls=":", color="black", label="Pseudocontinuum")
    # ax.fill_between([lambda_mid_c_A - EW / 2, lambda_mid_c_A + EW / 2], [y(lambda_mid_c_A - EW / 2), y(lambda_mid_c_A + EW / 2)], color="green", alpha=0.3)
    # ax.set_xlim([3900, 4200])
    # ax.set_ylim([0, None])
    # ax.set_xlabel(r"$\lambda$ (rest)")
    # ax.legend()
    
    return EW_A


In [6]:
###################################################
# Function for computing the D4000Å break strength
# as per Balogh (1999)
###################################################
def compute_D4000(spec, lambda_vals_A, z):

    # De-redshift the input spectrum
    lambda_vals_rest_A = lambda_vals_A / (1 + z) 

    # Compute the D4000Å break
    # Definition from Balogh+1999 (see here: https://arxiv.org/pdf/1611.07050.pdf, page 3)
    lambda_start_b_A = 3850
    lambda_stop_b_A = 3950
    lambda_start_r_A = 4000
    lambda_stop_r_A = 4100
    lambda_start_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_b_A))
    lambda_stop_b_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_b_A))
    lambda_start_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_start_r_A))
    lambda_stop_r_idx = np.nanargmin(np.abs(lambda_vals_rest_A - lambda_stop_r_A))
    N_b = lambda_stop_b_idx - lambda_start_b_idx
    N_r = lambda_stop_r_idx - lambda_start_r_idx

    # Convert datacube & variance cubes to units of F_nu
    F_lambda = np.copy(spec)
    F_nu = F_lambda * lambda_vals_rest_A**2 / (constants.c * 1e10)

    num = np.nanmean(F_nu[lambda_start_r_idx:lambda_stop_r_idx], axis=0)
    denom = np.nanmean(F_nu[lambda_start_b_idx:lambda_stop_b_idx], axis=0)

    d4000 = num / denom

    # ###################################################
    # # Plot
    # ###################################################
    # fig, ax = plt.subplots(figsize=(15, 5))
    # ax.plot(lambda_vals_rest_A, spec, color="grey")
    # ax.axvline(lambda_start_b_A, color="blue", label="Blue passband")
    # ax.axvline(lambda_stop_b_A, color="blue")
    # ax.axvline(lambda_start_r_A, color="red", label="Red passband")
    # ax.axvline(lambda_stop_r_A, color="red")
    # ax.set_ylim([0, None])
    # ax.set_xlabel(r"$\lambda$ (rest)")
    # ax.legend()
    
    return d4000

In [7]:
###################################################
# Function for evaluating "continuous" SFHs 
# as defined by Kauffmann+2003
###################################################
def sfr_fn(gamma, t_Gyr, t0_Gyr):
    """
    gamma = exponent (0-1)
    t0_Gyr = time since present day 
    
    """
    # Set SFR to be, let's say, 100 Msun/yr at t = t0_Gyr.
    sfr0 = 100
    sfr = sfr0 * np.exp( gamma * (t_Gyr - t0_Gyr))
    
    # Zero the SFR beyond t0_Gyr
    sfr[t_Gyr > t0_Gyr] = 0
    
    return sfr
    
# Plot the exponentially decaying SFR
t_Gyr = np.linspace(0, 15, 500)
fig, ax = plt.subplots()
ax.plot()
ax.plot(t_Gyr, sfr_fn(gamma=0.5, t_Gyr=t_Gyr, t0_Gyr=10), label=r"$\gamma = 0.5, t_0 = 10\,\rm Gyr$")
ax.plot(t_Gyr, sfr_fn(gamma=0.5, t_Gyr=t_Gyr, t0_Gyr=5), label=r"$\gamma = 0.5, t_0 = 5\,\rm Gyr$")
ax.plot(t_Gyr, sfr_fn(gamma=1.0, t_Gyr=t_Gyr, t0_Gyr=5), label=r"$\gamma = 1.0, t_0 = 5\,\rm Gyr$")

# Plot on our age grid to check
ax.step(ages / 1e9, sfr_fn(gamma=1.0, t_Gyr=ages / 1e9, t0_Gyr=5), ls="--", where="mid",
        label=r"SSP template ages ($\gamma = 1.0, t_0 = 5\,\rm Gyr$)")

ax.set_xlabel("Lookback time (Gyr)")
ax.set_ylabel(r"SFR ($\rm M_\odot \, yr^{-1}$)")
ax.legend()
ax.set_xscale("log")

# Plot the equivalent mass-weighted SFH to check 
sfh_mw = np.zeros((N_metallicities, N_ages))
sfh_mw[1, :] = sfr_fn(gamma=1.0, t_Gyr=ages / 1e9, t0_Gyr=5) * bin_widths
plot_sfh_mass_weighted(sfh_mw, ages=ages, metallicities=metallicities)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
df = pd.DataFrame()

In [9]:
##############################################################
# Compute Hdelta, D4000 break strength for a variety of SFHs
##############################################################
isochrones = "Padova"
SNR = 200
z = 0.01
sigma_star_kms = 250

# Instantaneous
for mm, aa in tqdm(product(range(N_metallicities), range(N_ages))):
    # Generate SFH
    sfh_mw = np.zeros((3, 74))
    sfh_mw[mm, aa] = 1e10
    sfh_lw = convert_mass_weights_to_light_weights(sfh_mw, isochrones="Padova")
    
    # Generate spectrum
    spec, spec_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw,
        agn_continuum=False,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)
    
    # Compute the mass-weighted mean age < 1 Gyr
    log_age_mw, _ = compute_mw_age(sfh_mw, isochrones=isochrones, age_thresh_upper=1e9)
    log_age_lw, _ = compute_lw_age(sfh_lw, isochrones=isochrones, age_thresh_upper=1e9)
        
    # Compute values, append to DataFrame
    df = df.append({
        "Isochrones": isochrones,
        "z": z,
        "sigma_* (km/s)": sigma_star_kms,
        "SNR": SNR,
        "SFH type": "instantaneous",
        "Z": metallicities[mm],
        "t_0 (yr)": ages[aa],
        "gamma": np.nan,
        "HdeltaA": compute_Hdelta(spec, lambda_vals_A, z),
        "D4000": compute_D4000(spec, lambda_vals_A, z),
        "AGN continuum": False,
        "MW age (< 1 Gyr)": 10**log_age_mw,
        "LW age (< 1 Gyr)": 10**log_age_lw,
    }, ignore_index=True)
    
# Continuous
for gamma, mm, aa in tqdm(product(np.linspace(0, 1, 4), range(N_metallicities), range(N_ages))):
    # Generate SFH
    sfh_mw = np.zeros((N_metallicities, N_ages))
    sfh_mw[mm, :] = sfr_fn(gamma=gamma, t_Gyr=ages / 1e9, t0_Gyr=ages[aa] / 1e9) * bin_widths
    sfh_lw = convert_mass_weights_to_light_weights(sfh_mw, isochrones="Padova")
    
    # Generate spectrum
    spec, spec_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw,
        agn_continuum=False,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)
    
    # Compute the mass-weighted mean age < 1 Gyr
    log_age_mw, _ = compute_mw_age(sfh_mw, isochrones=isochrones, age_thresh_upper=1e9)
    log_age_lw, _ = compute_lw_age(sfh_lw, isochrones=isochrones, age_thresh_upper=1e9)
    
    # Compute values, append to DataFrame
    df = df.append({
        "Isochrones": isochrones,
        "z": z,
        "sigma_* (km/s)": sigma_star_kms,
        "SNR": SNR,
        "SFH type": "continuous",
        "Z": metallicities[mm],
        "t_0 (yr)": ages[aa],
        "gamma": gamma,
        "HdeltaA": compute_Hdelta(spec, lambda_vals_A, z),
        "D4000": compute_D4000(spec, lambda_vals_A, z),
        "AGN continuum": False,
        "MW age (< 1 Gyr)": 10**log_age_mw,
        "LW age (< 1 Gyr)": 10**log_age_lw,
    }, ignore_index=True)



/home/u5708159/python/Modules/ppxftests/sfhutils.py:231: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:292: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


In [47]:
##############################################################
# Compute Hdelta, D4000 break strength for a variety of SFHs
# Add AGN continua
# Only compute for a single metallicity & a single SFH for now
##############################################################
isochrones = "Padova"
SNR = 200
z = 0.01
sigma_star_kms = 250

# AGN continuum parameters
alpha_nu_vals = np.linspace(0.3, 2.0, 4)
log_L_NT_vals = np.linspace(41, 44, 4)

# Fixed SFH parameters
mm = 1

# Instantaneous
for alpha_nu, log_L_NT in tqdm(product(alpha_nu_vals, log_L_NT_vals)):
    for aa in range(N_ages):
        # Generate SFH
        sfh_mw = np.zeros((3, 74))
        sfh_mw[mm, aa] = 1e10
        sfh_lw = convert_mass_weights_to_light_weights(sfh_mw, isochrones="Padova")

        # Generate spectrum
        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw,
            agn_continuum=True, alpha_nu=alpha_nu, L_NT_erg_s=10**log_L_NT,
            isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
            plotit=False)

        # Compute the mass-weighted mean age < 1 Gyr
        log_age_mw, _ = compute_mw_age(sfh_mw, isochrones=isochrones, age_thresh_upper=1e9)
        log_age_lw, _ = compute_lw_age(sfh_lw, isochrones=isochrones, age_thresh_upper=1e9)

        # Compute values, append to DataFrame
        df = df.append({
            "Isochrones": isochrones,
            "z": z,
            "sigma_* (km/s)": sigma_star_kms,
            "SNR": SNR,
            "SFH type": "instantaneous",
            "Z": metallicities[mm],
            "t_0 (yr)": ages[aa],
            "gamma": np.nan,
            "HdeltaA": compute_Hdelta(spec, lambda_vals_A, z),
            "D4000": compute_D4000(spec, lambda_vals_A, z),
            "AGN continuum": True,
            "alpha_nu": alpha_nu,
            "log L_NT": log_L_NT,
            "MW age (< 1 Gyr)": 10**log_age_mw,
            "LW age (< 1 Gyr)": 10**log_age_lw,
        }, ignore_index=True)
    
# Continuous
gamma = 1.0
for alpha_nu, log_L_NT in tqdm(product(alpha_nu_vals, log_L_NT_vals)):
    for aa in range(N_ages):
        # Generate SFH
        sfh_mw = np.zeros((N_metallicities, N_ages))
        sfh_mw[mm, :] = sfr_fn(gamma=gamma, t_Gyr=ages / 1e9, t0_Gyr=ages[aa] / 1e9) * bin_widths
        sfh_lw = convert_mass_weights_to_light_weights(sfh_mw, isochrones="Padova")

        # Generate spectrum
        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw,
            agn_continuum=True, alpha_nu=alpha_nu, L_NT_erg_s=10**log_L_NT,
            isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
            plotit=False)

        # Compute the mass-weighted mean age < 1 Gyr
        log_age_mw, _ = compute_mw_age(sfh_mw, isochrones=isochrones, age_thresh_upper=1e9)
        log_age_lw, _ = compute_lw_age(sfh_lw, isochrones=isochrones, age_thresh_upper=1e9)

        # Compute values, append to DataFrame
        df = df.append({
            "Isochrones": isochrones,
            "z": z,
            "sigma_* (km/s)": sigma_star_kms,
            "SNR": SNR,
            "SFH type": "continuous",
            "Z": metallicities[mm],
            "t_0 (yr)": ages[aa],
            "gamma": gamma,
            "HdeltaA": compute_Hdelta(spec, lambda_vals_A, z),
            "D4000": compute_D4000(spec, lambda_vals_A, z),
            "AGN continuum": True,
            "alpha_nu": alpha_nu,
            "log L_NT": log_L_NT,
            "MW age (< 1 Gyr)": 10**log_age_mw,
            "LW age (< 1 Gyr)": 10**log_age_lw,
        }, ignore_index=True)


In [22]:
df

,AGN continuum,D4000,HdeltaA,Isochrones,LW age (< 1 Gyr),MW age (< 1 Gyr),SFH type,SNR,Z,gamma,sigma_* (km/s),t_0 (yr),z
0,0.0,0.975599,2.832376,Padova,3.981100e+06,3.981100e+06,instantaneous,200.0,0.004,NaN,250.0,3.981100e+06,0.01
1,0.0,0.971296,2.874503,Padova,4.466800e+06,4.466800e+06,instantaneous,200.0,0.004,NaN,250.0,4.466800e+06,0.01
2,0.0,0.972783,2.904024,Padova,5.011900e+06,5.011900e+06,instantaneous,200.0,0.004,NaN,250.0,5.011900e+06,0.01
3,0.0,0.994049,3.225881,Padova,5.623400e+06,5.623400e+06,instantaneous,200.0,0.004,NaN,250.0,5.623400e+06,0.01
4,0.0,1.002153,3.661033,Padova,6.309600e+06,6.309600e+06,instantaneous,200.0,0.004,NaN,250.0,6.309600e+06,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,0.0,2.377350,-6.107149,Padova,1.625399e+08,4.382904e+08,continuous,200.0,0.019,1.0,250.0,1.122000e+10,0.01
1106,0.0,2.426348,-6.430418,Padova,1.625399e+08,4.382904e+08,continuous,200.0,0.019,1.0,250.0,1.258900e+10,0.01
1107,0.0,2.530223,-7.082934,Padova,1.625399e+08,4.382904e+08,continuous,200.0,0.019,1.0,250.0,1.412500e+10,0.01
1108,0.0,2.576294,-7.251045,Padova,1.625399e+08,4.382904e+08,continuous,200.0,0.019,1.0,250.0,1.584900e+10,0.01


In [17]:
df_gals = pd.DataFrame()

In [19]:
###################################################
# What do the simulated SFHs look like?
###################################################
for gal in tqdm(range(1132)):
    # Generate SFH
    try:
        sfh_mw, sfh_lw, sfr_avg, sigma_star_kms = load_sfh(gal=gal, plotit=False)
    except:
        print(f"Error occurred loading SFH for galaxy {gal}")
        continue

    # If the BH mass is listed as zero, then sigma_star_kms will also be 0 - in this case, set to a default value.
    if sigma_star_kms == 0:
        sigma_star_kms = 200
    
    # Generate spectrum
    try:
        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw,
            agn_continuum=False,
            isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
            plotit=False)
    except:
        print(f"Error occurred generating spectrum for galaxy {gal}")
        Tracer()()
        continue
    
    # Compute values, append to DataFrame
    df_gals = df_gals.append({
        "ID": gal,
        "Isochrones": isochrones,
        "z": z,
        "sigma_* (km/s)": sigma_star_kms,
        "SNR": SNR,
        "HdeltaA": compute_Hdelta(spec, lambda_vals_A, z),
        "D4000": compute_D4000(spec, lambda_vals_A, z),
        "AGN continuum": False,
        "MW age (< 1 Gyr)": compute_mw_age(sfh_mw, isochrones=isochrones, age_thresh_upper=1e9)[1],
        "LW age (< 1 Gyr)": compute_lw_age(sfh_lw, isochrones=isochrones, age_thresh_upper=1e9)[1]
    }, ignore_index=True)
    

/home/u5708159/python/Modules/ppxftests/sfhutils.py:231: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:292: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


Error occurred loading SFH for galaxy 359
Error occurred loading SFH for galaxy 399
Error occurred loading SFH for galaxy 460
Error occurred loading SFH for galaxy 469
Error occurred loading SFH for galaxy 481
Error occurred loading SFH for galaxy 510
Error occurred loading SFH for galaxy 520
Error occurred loading SFH for galaxy 525
Error occurred loading SFH for galaxy 527
Error occurred loading SFH for galaxy 528
Error occurred loading SFH for galaxy 530
Error occurred loading SFH for galaxy 535
Error occurred loading SFH for galaxy 538
Error occurred loading SFH for galaxy 542
Error occurred loading SFH for galaxy 549
Error occurred loading SFH for galaxy 553
Error occurred loading SFH for galaxy 554
Error occurred loading SFH for galaxy 555
Error occurred loading SFH for galaxy 558
Error occurred loading SFH for galaxy 560
Error occurred loading SFH for galaxy 562
Error occurred loading SFH for galaxy 565
Error occurred loading SFH for galaxy 570
Error occurred loading SFH for gal

Error occurred loading SFH for galaxy 832
Error occurred loading SFH for galaxy 834
Error occurred loading SFH for galaxy 835
Error occurred loading SFH for galaxy 836
Error occurred loading SFH for galaxy 837
Error occurred loading SFH for galaxy 839
Error occurred loading SFH for galaxy 840
Error occurred loading SFH for galaxy 841
Error occurred loading SFH for galaxy 842
Error occurred loading SFH for galaxy 844
Error occurred loading SFH for galaxy 846
Error occurred loading SFH for galaxy 847
Error occurred loading SFH for galaxy 848
Error occurred loading SFH for galaxy 849
Error occurred loading SFH for galaxy 850
Error occurred loading SFH for galaxy 851
Error occurred loading SFH for galaxy 852
Error occurred loading SFH for galaxy 853
Error occurred loading SFH for galaxy 854
Error occurred loading SFH for galaxy 855
Error occurred loading SFH for galaxy 856
Error occurred loading SFH for galaxy 857
Error occurred loading SFH for galaxy 858
Error occurred loading SFH for gal

Error occurred loading SFH for galaxy 1050
Error occurred loading SFH for galaxy 1051
Error occurred loading SFH for galaxy 1052
Error occurred loading SFH for galaxy 1053
Error occurred loading SFH for galaxy 1054
Error occurred loading SFH for galaxy 1055
Error occurred loading SFH for galaxy 1056
Error occurred loading SFH for galaxy 1057
Error occurred loading SFH for galaxy 1058
Error occurred loading SFH for galaxy 1059
Error occurred loading SFH for galaxy 1060
Error occurred loading SFH for galaxy 1061
Error occurred loading SFH for galaxy 1062
Error occurred loading SFH for galaxy 1063
Error occurred loading SFH for galaxy 1064
Error occurred loading SFH for galaxy 1065
Error occurred loading SFH for galaxy 1066
Error occurred loading SFH for galaxy 1067
Error occurred loading SFH for galaxy 1068
Error occurred loading SFH for galaxy 1069
Error occurred loading SFH for galaxy 1070
Error occurred loading SFH for galaxy 1071
Error occurred loading SFH for galaxy 1072
Error occur

In [60]:
###################################################
# Recreate fig. 3 of Kauffmann+2003
###################################################
z = 0
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.1, 0.5, 0.8])

gamma_vals = np.linspace(0, 1, 4)
m_colours = ["yellow", "orange", "red"]
g_linestyles = [":", "-.", "--", "--"]

# Instantaneous SFHs
for mm in range(N_metallicities):
    cond = df["SFH type"] == "instantaneous"
    cond &= df["AGN continuum"] == False
    cond &= df["Z"] == metallicities[mm]
    ax.plot(df.loc[cond, "D4000"], df.loc[cond, "HdeltaA"], 
            ls="-", color=m_colours[mm],
            label=r"Instantaneous ($Z = %.3f$)" % (metallicities[mm]))

# Continuous SFHs
for gg, mm in product(range(len(gamma_vals)), range(N_metallicities)):
    cond = df["SFH type"] == "continuous"
    cond &= df["AGN continuum"] == False
    cond &= df["Z"] == metallicities[mm]
    cond &= df["gamma"] == gamma_vals[gg]
    ax.plot(df.loc[cond, "D4000"], df.loc[cond, "HdeltaA"], 
            ls=g_linestyles[gg], color=m_colours[mm],
            label=r"Continuous ($Z = %.3f,\,\gamma = %.2f$)" % (metallicities[mm], gamma_vals[gg]))

# Over-plot galaxies
# ax.scatter(df_gals["D4000"], df_gals["HdeltaA"], c="k", marker=".")
    
ax.legend(loc="center left", bbox_to_anchor=[1.05, 0.5], fontsize="x-small")
ax.set_xlabel(r"$\rm D_n(4000)$")
ax.set_ylabel(r"H$\delta_{\rm A}$")
ax.set_xlim([0.7, 2.5])
ax.set_ylim([-6, 12])
ax.grid()


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [124]:
###################################################
# Compute these values for the S7 sample
###################################################

fits_path = "/priv/meggs3/u5708159/S7/4_Nuclear_spectra_Re/"
gals = [fname.split("_COMB_Re.fits")[0] for fname in os.listdir(fits_path) if fname.endswith(".fits") and not fname.startswith("._")]

df_s7 = pd.DataFrame(index=gals)
df_s7.index.name = "ID"

for gal in tqdm(gals[:5]):
    # Open FITS file
    hdulist = fits.open(os.path.join(fits_path, f"{gal}_COMB_Re.fits"))
    spec = hdulist[0].data
    spec_err = hdulist[1].data
    z = hdulist[0].header["Z"]
    
    # Wavelength information
    N_lambda = hdulist[0].header["NAXIS1"]
    dlambda_A = hdulist[0].header["CDELT1"]
    lambda_start_A = hdulist[0].header["CRVAL1"]
    lambda_vals_A = np.array(range(N_lambda)) * dlambda_A + lambda_start_A
    
    # Run ppxf to fit emission lines 
    pp = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=2,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=True, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    
    # Compute the index strength from the ppxf continuum fit
    spec_cont_log = (pp.bestfit - pp.gas_bestfit) / pp.mpoly
    lambda_vals_log_A = pp.lam
    
    df_s7.loc[gal, "D4000"] = compute_D4000(spec_cont_log, lambda_vals_log_A, z)
    df_s7.loc[gal, "HdeltaA"] = compute_Hdelta(spec_cont_log, lambda_vals_log_A, z)
    
    

Emission lines included in gas templates:
['Balmer' '[OII]3726' '[OII]3729' '[SII]6716' '[SII]6731' '[OIII]5007_d'
 '[OI]6300_d' '[NII]6583_d' '[NeIII]3869' 'HeI3889']
Emission lines included in gas templates:
['Balmer' '[OII]3726' '[OII]3729' '[SII]6716' '[SII]6731' '[OIII]5007_d'
 '[OI]6300_d' '[NII]6583_d' '[NeIII]3869' 'HeI3889']
Emission lines included in gas templates:
['Balmer' '[OII]3726' '[OII]3729' '[SII]6716' '[SII]6731' '[OIII]5007_d'
 '[OI]6300_d' '[NII]6583_d' '[NeIII]3869' 'HeI3889']
Emission lines included in gas templates:
['Balmer' '[OII]3726' '[OII]3729' '[SII]6716' '[SII]6731' '[OIII]5007_d'
 '[OI]6300_d' '[NII]6583_d' '[NeIII]3869' 'HeI3889']
Emission lines included in gas templates:
['Balmer' '[OII]3726' '[OII]3729' '[SII]6716' '[SII]6731' '[OIII]5007_d'
 '[OI]6300_d' '[NII]6583_d' '[NeIII]3869' 'HeI3889']



In [122]:
plt.figure()
plt.plot(pp.lam, pp.gas_bestfit)
plt.plot(pp.lam, pp.bestfit)
plt.plot(pp.lam, (pp.bestfit - pp.gas_bestfit) / pp.mpoly)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
plt.figure();
plt.plot(pp.lam, pp.mpoly)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [125]:
###################################################
# What is the effect of an AGN continuum?
###################################################
df_agn = df[df["AGN continuum"] == True]
met = df_agn["Z"].unique()[0]
alpha_nu_vals = df_agn["alpha_nu"].unique()
log_L_NT_vals = df_agn["log L_NT"].unique()

cmap_alpha_nu = plt.cm.get_cmap("jet", len(alpha_nu_vals) + 1)
cmap_log_L_NT = plt.cm.get_cmap("viridis", len(log_L_NT_vals) + 1)
lw_vals = np.linspace(0.5, 2.5, len(log_L_NT_vals))

# Instantaneous SFHs
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.1, 0.5, 0.8])

# Plot the "control"
cond = df["SFH type"] == "instantaneous"
cond &= df["Z"] == met
cond &= df["AGN continuum"] == False
ax.plot(df.loc[cond, "D4000"], df.loc[cond, "HdeltaA"], 
        ls="--", color="k",
        label=r"Instantaneous (no AGN continuum)")

# Plot the locus with the AGN continuum
jj = 1
ii = 0
for ii in range(len(alpha_nu_vals)):
    for jj in range(len(log_L_NT_vals)):
        cond = df_agn["SFH type"] == "instantaneous"
        cond &= df_agn["Z"] == met
        cond &= df_agn["alpha_nu"] == alpha_nu_vals[ii] 
        cond &= df_agn["log L_NT"] == log_L_NT_vals[jj] 
        ax.plot(df_agn.loc[cond, "D4000"].values[:74], df_agn.loc[cond, "HdeltaA"].values[:74], 
                ls="-", color=cmap_alpha_nu(ii), linewidth=lw_vals[jj],
                label=r"Instantaneous ($\alpha_\nu = %.3f,\, \log_{10} L_{\rm NT} = %.2f$)" % (alpha_nu_vals[ii], log_L_NT_vals[jj]))

plt.scatter(df_s7["D4000"], df_s7["HdeltaA"], marker="^", c="b", label="S7 galaxies")
        
ax.legend(loc="center left", bbox_to_anchor=[1.05, 0.5], fontsize="x-small")
ax.set_xlabel(r"$\rm D_n(4000)$")
ax.set_ylabel(r"H$\delta_{\rm A}$")
ax.set_xlim([0.7, 2.5])
ax.set_ylim([-6, 12])
ax.grid()

# continuous SFHs
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.1, 0.5, 0.8])

gamma = df_agn.loc[df_agn["SFH type"] == "continuous", "gamma"].unique()

# Plot the "control"
cond = df["SFH type"] == "continuous"
cond &= df["Z"] == met
cond &= df["AGN continuum"] == False
ax.plot(df.loc[cond, "D4000"].values[:74], df.loc[cond, "HdeltaA"].values[:74], 
        ls="--", color="k",
        label=r"Continuous, $\gamma = %.1f$ (no AGN continuum)" % gamma)

# Plot the locus with the AGN continuum
for ii in range(len(alpha_nu_vals)):
    for jj in range(len(log_L_NT_vals)):
        cond = df_agn["SFH type"] == "continuous"
        cond &= df_agn["Z"] == met
        cond &= df_agn["alpha_nu"] == alpha_nu_vals[ii] 
        cond &= df_agn["log L_NT"] == log_L_NT_vals[jj] 
        ax.plot(df_agn.loc[cond, "D4000"].values[:74], df_agn.loc[cond, "HdeltaA"].values[:74], 
                ls="-", color=cmap_alpha_nu(ii), linewidth=lw_vals[jj],
                label=r"Continuous ($\alpha_\nu = %.3f,\, \log_{10} L_{\rm NT} = %.2f$)" % (alpha_nu_vals[ii], log_L_NT_vals[jj]))

plt.scatter(df_s7["D4000"], df_s7["HdeltaA"], marker="^", c="b", label="S7 galaxies")        
        
ax.legend(loc="center left", bbox_to_anchor=[1.05, 0.5], fontsize="x-small")
ax.set_xlabel(r"$\rm D_n(4000)$")
ax.set_ylabel(r"H$\delta_{\rm A}$")
ax.set_xlim([0.7, 2.5])
ax.set_ylim([-6, 12])
ax.grid()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
###################################################
# Recreate fig. 3 of Kauffmann+2003
# Scatter plot coloured by MW/LW ages
###################################################
z = 0
fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0.1, 0.1, 0.5, 0.8])
cax = fig.add_axes([0.6, 0.1, 0.025, 0.8])

gamma_vals = np.linspace(0, 1, 4)
m_colours = ["yellow", "orange", "red"]
g_linestyles = [":", "-.", "--", "--"]

# Instantaneous SFHs
for mm in range(N_metallicities):
    cond = df["SFH type"] == "instantaneous"
    cond &= df["AGN continuum"] == False
    cond &= df["Z"] == metallicities[mm]
    m = ax.scatter(x=df.loc[cond, "D4000"], y=df.loc[cond, "HdeltaA"], 
               c=np.log10(df.loc[cond, "MW age (< 1 Gyr)"]), cmap="Spectral_r", 
               marker="o", s=5, vmin=6, vmax=9, alpha=0.4,
               label=r"Instantaneous ($Z = %.3f$)" % (metallicities[mm]))

# Continuous SFHs
for gg, mm in product(range(len(gamma_vals)), range(N_metallicities)):
    cond = df["SFH type"] == "continuous"
    cond &= df["AGN continuum"] == False
    cond &= df["Z"] == metallicities[mm]
    cond &= df["gamma"] == gamma_vals[gg]
    m = ax.scatter(x=df.loc[cond, "D4000"], y=df.loc[cond, "HdeltaA"], 
               c=np.log10(df.loc[cond, "MW age (< 1 Gyr)"]), cmap="Spectral_r", 
               marker="o", s=5, vmin=6, vmax=9, alpha=0.4,
               label=r"Continuous ($Z = %.3f,\,\gamma = %.2f$)" % (metallicities[mm], gamma_vals[gg]))
    
# Over-plot galaxies
ax.scatter(x=df_gals["D4000"], y=df_gals["HdeltaA"], 
           c=np.log10(df_gals["MW age (< 1 Gyr)"]), 
           cmap="Spectral_r", marker="o")
    
# Decorations
plt.colorbar(mappable=m, cax=cax)
ax.legend(loc="center left", bbox_to_anchor=[1.05, 0.5], fontsize="x-small")
ax.set_xlabel(r"$\rm D_n(4000)$")
ax.set_ylabel(r"H$\delta_{\rm A}$")
ax.set_xlim([0.7, 2.5])
ax.set_ylim([-6, 12])
ax.grid()


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …